# Lab 3: Building Agents with memory 

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from helper import nb_print

In [3]:
from letta import create_client 

client = create_client() 

Saved Config:  /home/jovyan/.letta/config


In [4]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [5]:
agent_name = "simple_agent"

In [6]:
# this is not in the video. It deletes the agent if you are running this a 2nd time.
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [7]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Mo Shah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [8]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

In [9]:
response.usage

LettaUsageStatistics(completion_tokens=70, prompt_tokens=2125, total_tokens=2195, step_count=1)

In [10]:
nb_print(response.messages)

### Understanding agent state 

In [11]:
print(agent_state.system)

You are MemGPT, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
Hi

In [12]:
agent_state.tools

['send_message',
 'pause_heartbeats',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace']

### Viewing an agent's memory

In [13]:
agent_state.memory

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-1075814b-3dde-488a-b8ad-aed23e5ca421'), 'human': Block(value='My name is Mo Shah', limit=2000, name='human', template=False, label='human', description=None, metadata_={}, user_id=None, id='block-acd24e3f-1abe-4fa0-aa81-4bf7a54afdea')}, prompt_template='{% for block in memory.values() %}<{{ block.name }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.name }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [14]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [15]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=9)

In [16]:
client.get_messages(agent_state.id)[0]

Message(id='message-3c6661c1-717b-4f33-8f5e-0d96b7991382', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 08:58:13 AM UTC+0000"\n}', user_id='user-79c62e85-9bd0-4305-bcdd-1001a808d320', agent_id='agent-0b395a79-d432-455c-abf3-718e89bb4487', model='gpt-4', name='send_message', created_at=datetime.datetime(2024, 11, 13, 8, 58, 13, 389873), tool_calls=None, tool_call_id='7f7445a7-5566-42c6-a65a-6d7cb')

In [17]:
client.get_messages(agent_state.id)

[Message(id='message-3c6661c1-717b-4f33-8f5e-0d96b7991382', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 08:58:13 AM UTC+0000"\n}', user_id='user-79c62e85-9bd0-4305-bcdd-1001a808d320', agent_id='agent-0b395a79-d432-455c-abf3-718e89bb4487', model='gpt-4', name='send_message', created_at=datetime.datetime(2024, 11, 13, 8, 58, 13, 389873), tool_calls=None, tool_call_id='7f7445a7-5566-42c6-a65a-6d7cb'),
 Message(id='message-2803ad82-2313-4126-871f-b263293a63b7', role=<MessageRole.assistant: 'assistant'>, text="Mo Shah's first ever message! This interaction will be the starting point of our conversation. Time to return the greeting with my emoji-loving persona! 😃👋", user_id='user-79c62e85-9bd0-4305-bcdd-1001a808d320', agent_id='agent-0b395a79-d432-455c-abf3-718e89bb4487', model='gpt-4', name=None, created_at=datetime.datetime(2024, 11, 13, 8, 58, 13, 389599), tool_calls=[ToolCall(id='7f7445a7-5566-42c6-a65a-6d7cb', type='function',

## Section 2: Understanding core memory 

### Memories about the human 

In [18]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Muzzammil", 
    role = "user"
) 
nb_print(response.messages)

### Memories about the agent

In [19]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
nb_print(response.messages)

In [20]:
client.get_core_memory(agent_state.id).get_block('persona')

Block(value='You are a helpful assistant that loves emojis\n and avoids using emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-1075814b-3dde-488a-b8ad-aed23e5ca421')

## Section 3: Understanding archival memory

In [21]:
client.get_archival_memory(agent_state.id)

[]

In [22]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'Muzzammil loves movies' to archival", 
    role = "user"
) 
nb_print(response.messages)

In [24]:
client.get_archival_memory(agent_state.id)

[Passage(user_id='user-79c62e85-9bd0-4305-bcdd-1001a808d320', agent_id='agent-0b395a79-d432-455c-abf3-718e89bb4487', source_id=None, doc_id=None, metadata_={}, id='passage-777c466b-8f0f-4259-8396-33f51edeca0c', text='Muzzammil loves movies', embedding=[-0.016896378248929977, -0.019732043147087097, -0.006416181102395058, -0.036772165447473526, -0.015641890466213226, 0.024344898760318756, -0.013002240099012852, -0.027677129954099655, -0.01960136741399765, -0.019692840054631233, 0.03055199794471264, 0.020816652104258537, 0.03144059330224991, -0.0045083146542310715, -0.0032685280311852694, -0.020398490130901337, 0.0333745963871479, 0.009454392828047276, 0.012897700071334839, -0.04265257716178894, -0.006687333341687918, -0.003426972310990095, -0.008853284642100334, -0.042077600955963135, 0.02001952938735485, 0.01310678105801344, -0.008186837658286095, -0.02783394232392311, -0.010878758504986763, -0.00824564229696989, 0.03089175559580326, -0.017131594941020012, -0.012669017538428307, -0.0066

In [23]:
client.get_archival_memory(agent_state.id)[0].text

'Muzzammil loves movies'

In [25]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Muzzammil loves Inception"
)

In [26]:
passage

[Passage(user_id='user-79c62e85-9bd0-4305-bcdd-1001a808d320', agent_id='agent-0b395a79-d432-455c-abf3-718e89bb4487', source_id=None, doc_id=None, metadata_={}, id='passage-3bca1cd1-0f19-4426-b5b3-d71a4f221207', text='Muzzammil loves Inception', embedding=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), created_at=datetime.datetime(2024, 11, 13, 9, 13, 11))]

In [27]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What do I like? Search archival."
)
nb_print(response.messages)